# EXTRACCIÓN DE LOS DATOS DE TELEGRAM

Para la red social de Telegram, se extraerán datos con dos scripts diferentes.<br>
- Script para sacar información de los perfiles e introducirlos en la BBDD
- Script para sacar información de los mensajes e introducirlos en la BBDD

## PSEUDOCÓDIGO

**El pseudocódigo del programa será de la siguiente manera**

Abrir Base de Datos
Seleccionar y extraer los datos de la tabla donde se almacenan los enlaces de los diferentes enlaces a los canales de Telegram<br><br>
Extraer los enlaces de telegram<br><br>
Para cada una de las cuentas:<br>
   &emsp; extraer los datos de la cuentas<br>
   &emsp; introducir los datos en la tabla correspondiente<br><br>
   &emsp; Para cada uno de los mensajes de la cuenta:<br>
   &emsp; &emsp; extraer los datos de los mensajes<br>
   &emsp; &emsp; introducir los datos en la tabla correspondiente<br>

## LIBRERÍAS

Para la correcta extracción y recopilación de los datos, se aplicarán diferentes librerías pero la librería principal es:<br><br>
**__SNSCRAPE__**: Librería desarrollada por el usuario de Github JustAnotherArchivist que tiene por misión facilitar la extracción de datos de diferentes Redes Sociales. En este caso en concreto se utilizará el módulo de Telegram, que se puede consultar __[aquí](https://github.com/JustAnotherArchivist/snscrape/blob/master/snscrape/modules/telegram.py)__. Como no tiene documentación, hay que leer y entender el código para su posterior aplicación.<br><br>

Otras librerías que se usan son:
- **Pandas** : Manipulación de datos
- **Mysql.connector**: Conexión con la base de datos
- **Codecs**: Normalización de caracteres


## PASOS PREVIOS

Para realizar estos scripts, primeramente se añaden dos tablas a la BBDD con la siguiente sintaxis:

- **TABLA PARA PERFILES DE TELEGRAM**:

CREATE TABLE IF NOT EXISTS canales_telegram(<br>
    &emsp; Nombre VARCHAR(100),<br>
	&emsp; Titulo VARCHAR(150),<br>
    &emsp; Suscritos INT,<br>
    &emsp; Fotos INT,<br>
    &emsp; Videos INT,<br>
    &emsp; Documentos INT,<br>
    &emsp; Enlances INT<br>
)ENGINE = INNODB;<br>



- **TABLA PARA LOS MENSAJES DE TELEGRAM**:

CREATE TABLE IF NOT EXISTS mensajes_canales_telegram(<br>
    &emsp; Nombre VARCHAR(100),<br>
    &emsp; Url TEXT,<br>
    &emsp; Fecha TIMESTAMP,<br>
    &emsp; Contenido TEXT,<br>
    &emsp; Enlaces INT<br>
)ENGINE = INNODB;<br>



## CÓDIGO

Ya que se tiene todo los requisitos, ahora se abre la conexión con la Base de Datos y se empieza a codificar. <br>
Lo primero de todo es importar las librerias que necesitamos

In [ ]:
#pip install snscrape

In [ ]:
import snscrape.modules.twitter as snst
import pandas as pd
import mysql.connector
import codecs

Acto seguido, se abre la conexión de la base de Datos con Python 

In [ ]:
database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

Se extraen las cuentas de Twitter de la tabla de la Base de Datos **"universidades"**

In [ ]:
datos = pd.read_sql("SELECT * FROM universidades",database)
cuentas_telegram = datos["Telegram"].tolist()
print(datos.info())

Como no todas las universidades tienen cuenta de telegram, se pasará a limpiar la lista.

In [ ]:
for a in range(len(cuentas_telegram)-1,-1,-1):
    if cuentas_telegram[a] == None or objeto == "":
        cuentas_telegram.pop(a)
print(cuentas_telegram, len(cuentas_telegram))
        

Conociendo estos, datos y haciendo un analisis de relación tiempo- esfuerzo, se decide recopilar los datos de los perfiles de manera manual mientras se ejecuta el script que recoge los mensajes. De esta manera se ahorra escribir y ejecutar un script.

In [ ]:
## EXTRACCIÓN Y PROCESAMIENTO DE MENSAJES DE TELEGRAM##
for titulo in cuentas_telegram:
    print(titulo)
    for i,post in enumerate(snstel.TelegramChannelScraper(titulo).get_items()):
        fecha = post.date.strftime('%Y-%m-%d %H:%M:%S')
        links = " , ".join(post.outlinks)
        post_info = (
                titulo,
                post.url,
                fecha,
                post.content,
                len(links))
        cursor.execute("INSERT INTO mensajes_canales_telegram VALUES(%s,%s,%s,%s,%s)",post_info)
        database.commit()

### Con este pequeño script están los datos de la red social Twitter extraidos.<br><br> Duración aproximada 3h